## Loading and saving a pre trained model.

This has to be replaced with your own h5 model.

In [1]:
from tensorflow import keras
import tensorflow

In [2]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input

In [3]:
# store the inception model in local as an h5 file.

inception_model = InceptionV3(weights='imagenet', input_tensor=Input(shape=(224, 224, 3)))
inception_model.save('inception.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
96116736/96112376 [==============================] - 1s 0us/step


### Convert the h5 model into pb format

In [4]:
import os 
import tensorflow as tf 
from tensorflow.keras import backend as K

In [5]:
# convert the model to pb format to deploy using tf server

export_path = './tfm/1'

with tf.keras.backend.get_session() as sess:
    tf.saved_model.simple_save(
        sess,
        export_path,
        inputs={'input_image': inception_model.input},
        outputs={t.name:t for t in inception_model.outputs})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./tfm/1/saved_model.pb


### Add the model files in a tar.gz file.

In [6]:
#compress model to tar.gz

import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('tfm/', recursive=True)


### upload model to s3

In [7]:

import sagemaker

sagemaker_session = sagemaker.Session()
model_path_s3 = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

In [8]:
model_path_s3

's3://sagemaker-us-west-1-365792799466/model/model.tar.gz'

### Create a Model using tensorflow serving and deploy it.

In [9]:
from sagemaker import get_execution_role,session
import boto3

role = get_execution_role()
sm_session = session.Session(boto3.Session())

In [10]:
from sagemaker.tensorflow.serving import Model

In [11]:
tfmodel = Model(model_data=model_path_s3,
                         role=role,
                         entry_point="inference.py",
                         source_dir="code",
                         framework_version='1.15',
                         sagemaker_session=sm_session)

In [12]:
predictor = tfmodel.deploy(initial_instance_count=1,
                                          instance_type='ml.t2.medium')

-----------------!

### Code to invoke the endpoint with an image

In [15]:
import json
import numpy as np
file_path = "dog.jpeg"

#this can be obtained from the sagemaker console under endpoints.
endpoint_name = "tensorflow-inference-2021-03-04-02-23-24-797"

sm_runtime = boto3.Session().client("sagemaker-runtime")

with open(file_path, "rb") as image:
    f = image.read()
    b = bytearray(f)
    resp = sm_runtime.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType='application/x-image',
                                   Body=b)
    predictions = json.loads(resp['Body'].read().decode('utf-8'))
    preds = np.array(predictions['predictions'][0])
    
    print("classification type: ",np.argmax(preds))

classification type:  898


In [16]:
! tar cvf tf_deploy.tar.gz .

/bin/sh: 1: zip: not found
